# **Tweepy API**

In [ ]:
pip install tweepy

In [ ]:
import tweepy
import json
import csv
import pandas as pd
from datetime import timedelta

# Authenticate to Twitter
auth = tweepy.OAuthHandler("KD8NW5hyiGCIhwp4JDJ98pgXh", "SQeokZiYEb5nqWiRMzwRIWF2QQyeBANeHvyfYg3BjenCp2T5vV")
auth.set_access_token("93126766-LZ166ZBBYttZ4JM5WKg9cLLBq0eoJyA0QJwq7l30V", "BlyNL7pReqLDQx0uWjNkMeXlg4w2j2U7cFFpPE1ATy29X")
api = tweepy.API(auth)

Collecting tweets

In [ ]:
import datetime

def import_tweet():
  cursor = tweepy.Cursor(api.search, q=('jalan bolong'), lang="id", tweet_mode="extended").items(450)
  csvFile = open('jokowi gimana.csv', 'w')
  csvWriter = csv.writer(csvFile)
  csvWriter.writerow(["Timestamp", "ID", "Username", "Text"])
  for tweet in cursor:
    now = datetime.datetime.now()
    remain = now - tweet.created_at
    secs = remain.total_seconds()
    if (secs < 7200):
      if (not tweet.retweeted) and ('RT @' not in tweet.full_text):
        print(tweet)
        tweets = [tweet.created_at + timedelta(hours=7),tweet.id,tweet.user.screen_name,tweet.full_text]
        csvWriter.writerow(tweets)

  csvFile.close()
  csvFile = open('jokowi gimana.csv', 'a')
  csvWriter = csv.writer(csvFile)
  cursor = tweepy.Cursor(api.search, q=('air mati'), lang="id", tweet_mode="extended").items(450)
  for tweet in cursor:
    now = datetime.datetime.now()
    remain = now - tweet.created_at
    secs = remain.total_seconds()
    if (secs < 7200):
      if (not tweet.retweeted) and ('RT @' not in tweet.full_text):
        print(tweet)
        tweets = [tweet.created_at + timedelta(hours=7),tweet.id,tweet.user.screen_name,tweet.full_text]
        csvWriter.writerow(tweets)

# **Preprocessing**

In [ ]:
pip install Sastrawi

     |████████████████████████████████| 215kB 6.8MB/s 


In [ ]:
pip install swifter

In [ ]:
import pandas as pd
import numpy as np
import re
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import swifter
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
dataTweet = pd.read_csv('CleanDataset.csv')

In [ ]:
dataTweet['Text']

0       Di Pepo mangkrak bangun jalan Non tol ?\nOtak ...
1       Pak ganjar hebat. Tp cuma untuk daerah2 yang p...
2       Kalo perbatasan bekasi jakarta mah satunya jal...
3       Naik motor kena jalan gejlukan jalan yg rusak ...
4       Bangun tidur badan sakit sakit semuaaa grgr ja...
                              ...                        
1768     mencari sinyal dari bts yang lebih jauh dan t...
1769     siang kak johan maaf ya mengenai keluhan siny...
1770    nyala juga ni listrik  bangun pagi gara listri...
1771    padam listrik berarti allah suruh lanjut rebah...
1772    mau isi sopipay aja susah banget sih  yang ero...
Name: Text, Length: 1773, dtype: object

Cleaning Tweet

In [ ]:
def processTweet(tweet):
    #Convert to lower case
    tweet = tweet.lower()
    #Remove non ASCII (emoticon, chinese word, .etc)
    tweet = tweet.encode('ascii', 'replace').decode('ascii')
    #Remove www.* or https?://* or http?://
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))','',tweet)
    #Remove @username
    tweet = re.sub('@[^\s]+','',tweet)
    #Remove additional white spaces
    tweet = " ".join(re.split(r"\s+", tweet))
    tweet = re.sub('[\s]+', ' ', tweet)
    #Remove number
    tweet = re.sub(r"\d+", "", tweet)
    #Remove single char
    tweet = re.sub(r'\b[a-zA-Z]\b', '', tweet)
    #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    #Remove punctuation
    tweet = tweet.translate(str.maketrans("", "", string.punctuation))
    #trim
    tweet = tweet.strip('\'"')
    return tweet

In [ ]:
dataTweet['TextProcessed'] = dataTweet['Text'].apply(processTweet)

In [ ]:
dataTweet

,Label,Text,TextProcessed
0,1,Di Pepo mangkrak bangun jalan Non tol ?\nOtak ...,di pepo mangkrak bangun jalan non tol otak mn...
1,1,Pak ganjar hebat. Tp cuma untuk daerah2 yang p...,pak ganjar hebat tp cuma untuk daerah yang per...
2,1,Kalo perbatasan bekasi jakarta mah satunya jal...,kalo perbatasan bekasi jakarta mah satunya jal...
3,1,Naik motor kena jalan gejlukan jalan yg rusak ...,naik motor kena jalan gejlukan jalan yg rusak ...
4,1,Bangun tidur badan sakit sakit semuaaa grgr ja...,bangun tidur badan sakit sakit semuaaa grgr ja...
...,...,...,...
1768,0,mencari sinyal dari bts yang lebih jauh dan t...,mencari sinyal dari bts yang lebih jauh dan t...
1769,0,siang kak johan maaf ya mengenai keluhan siny...,siang kak johan maaf ya mengenai keluhan siny...
1770,0,nyala juga ni listrik bangun pagi gara listri...,nyala juga ni listrik bangun pagi gara listrik...
1771,0,padam listrik berarti allah suruh lanjut rebah...,padam listrik berarti allah suruh lanjut rebah...


Stopwords

In [ ]:
#Get stopword indonesia
list_stopwords = stopwords.words('indonesian')

#Append additional stopword
list_stopwords.extend(['yg', 'dg', 'rt', 'dgn', 'ny', 'd', 'klo', 'tsb', 'sy','sya', 'gw', 'gue', 'gua', 'eug', 'aq', 'akyu', 'km', 'kmu', 'dih',
                       'kalo', 'amp', 'biar', 'bikin', 'bilang', 'dongg', 'pol', 'poll', 'pooll', 'iy', 'y', 'yahh', 'kta', 'weh', 'widiw', 'idih',
                       'gak', 'ga', 'g', 'krna', 'mw', 'krn', 'krna', 'nya', 'nyaa', 'nih', 'sih', 'noh', 'nah', 'neh', 'fess', 'wadaw', 'wadidaw',
                       'si', 'tau', 'tdk', 'tuh', 'utk', 'ya', 'tw', 'tao', 'mao', 'geh', 'gih', 'min', 'mimin', 'guys', 'ges', 'gaes', 'bgmn',
                       'jd', 'jgn', 'sdh', 'aja', 'n', 't', 'sj', 'sja', 'dong', 'klok', 'anjir', 'njir', 'jir', 'bjir', 'ap', 'knp', 'knapa', 'gmn', 'uwu',
                       'nyg', 'hehe', 'haha', 'pen', 'u', 'nan', 'loh', 'rt', 'wkwk', 'wkwkwk', 'hahaha', 'xixixi', 'jajaja', 'dsni', 'kk', 'kaka', 'huhu',
                       '&amp', 'yah', 'klean', 'kpn', 'ih', 'ah', 'eh', 'oh', 'oap', 'iye', 'uy', 'cuy', 'bzz'])

# convert list to dictionary
list_stopwords = set(list_stopwords)

def remove_stopwords(tweet):
  tweets = [word for word in tweet.split() if word not in list_stopwords]
  return tweets

In [ ]:
dataTweet['TextProcessed'] = dataTweet['TextProcessed'].swifter.apply(remove_stopwords)

In [ ]:
dataTweet

,Label,Text,TextProcessed
0,1,Di Pepo mangkrak bangun jalan Non tol ?\nOtak ...,"[pepo, mangkrak, bangun, jalan, non, tol, otak..."
1,1,Pak ganjar hebat. Tp cuma untuk daerah2 yang p...,"[ganjar, hebat, tp, daerah, perkotaan, daerah,..."
2,1,Kalo perbatasan bekasi jakarta mah satunya jal...,"[perbatasan, bekasi, jakarta, mah, satunya, ja..."
3,1,Naik motor kena jalan gejlukan jalan yg rusak ...,"[motor, kena, jalan, gejlukan, jalan, rusak, n..."
4,1,Bangun tidur badan sakit sakit semuaaa grgr ja...,"[bangun, tidur, badan, sakit, sakit, semuaaa, ..."
...,...,...,...
1768,0,mencari sinyal dari bts yang lebih jauh dan t...,"[mencari, sinyal, bts, kondisi, sinyal, lemah,..."
1769,0,siang kak johan maaf ya mengenai keluhan siny...,"[siang, kak, johan, maaf, keluhan, sinyal, hil..."
1770,0,nyala juga ni listrik bangun pagi gara listri...,"[nyala, ni, listrik, bangun, pagi, gara, listr..."
1771,0,padam listrik berarti allah suruh lanjut rebah...,"[padam, listrik, allah, suruh, rebahan]"


Normalization

In [ ]:
normalized_word = pd.read_excel("normalization.xlsx")

normalized_word_dict = {}

for index, row in normalized_word.iterrows():
    if row[0] not in normalized_word_dict:
        normalized_word_dict[row[0]] = row[1] 

def normalized_term(tweet):
    return [normalized_word_dict[term] if term in normalized_word_dict else term for term in tweet]

In [ ]:
dataTweet['TextProcessed'] = dataTweet['TextProcessed'].swifter.apply(normalized_term)

In [ ]:
dataTweet

,Label,Text,TextProcessed
0,1,Di Pepo mangkrak bangun jalan Non tol ?\nOtak ...,"[pepo, mangkrak, bangun, jalan, non, tol, otak..."
1,1,Pak ganjar hebat. Tp cuma untuk daerah2 yang p...,"[ganjar, hebat, tapi, daerah, perkotaan, daera..."
2,1,Kalo perbatasan bekasi jakarta mah satunya jal...,"[perbatasan, bekasi, jakarta, mah, satunya, ja..."
3,1,Naik motor kena jalan gejlukan jalan yg rusak ...,"[motor, kena, jalan, gejlukan, jalan, rusak, s..."
4,1,Bangun tidur badan sakit sakit semuaaa grgr ja...,"[bangun, tidur, badan, sakit, sakit, semua, ga..."
...,...,...,...
1768,0,mencari sinyal dari bts yang lebih jauh dan t...,"[mencari, sinyal, bts, kondisi, sinyal, lemah,..."
1769,0,siang kak johan maaf ya mengenai keluhan siny...,"[siang, kak, johan, maaf, keluhan, sinyal, hil..."
1770,0,nyala juga ni listrik bangun pagi gara listri...,"[nyala, ini, listrik, bangun, pagi, gara, list..."
1771,0,padam listrik berarti allah suruh lanjut rebah...,"[padam, listrik, allah, suruh, rebahan]"


Stemming

In [ ]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stemmed_wrapper(tweets):
    return stemmer.stem(tweets)

In [ ]:
word_dict = {}

In [ ]:
for tweet in dataTweet['TextProcessed']:
    for word in tweet:
        if word not in word_dict:
            word_dict[word] = ' '            

for word in word_dict:
    word_dict[word] = stemmed_wrapper(word)

def get_stemmed_term(tweet):
  words = [word_dict[word] for word in tweet]
  return words

In [ ]:
dataTweet['TextProcessed'] = dataTweet['TextProcessed'].swifter.apply(get_stemmed_term)

In [ ]:
dataTweet

,Label,Text,TextProcessed
0,1,Di Pepo mangkrak bangun jalan Non tol ?\nOtak ...,"[pepo, mangkrak, bangun, jalan, non, tol, otak..."
1,1,Pak ganjar hebat. Tp cuma untuk daerah2 yang p...,"[ganjar, hebat, tapi, daerah, kota, daerah, ba..."
2,1,Kalo perbatasan bekasi jakarta mah satunya jal...,"[batas, bekas, jakarta, mah, satu, jalan, rusa..."
3,1,Naik motor kena jalan gejlukan jalan yg rusak ...,"[motor, kena, jalan, gejlukan, jalan, rusak, s..."
4,1,Bangun tidur badan sakit sakit semuaaa grgr ja...,"[bangun, tidur, badan, sakit, sakit, semua, ga..."
...,...,...,...
1768,0,mencari sinyal dari bts yang lebih jauh dan t...,"[cari, sinyal, bts, kondisi, sinyal, lemah, ba..."
1769,0,siang kak johan maaf ya mengenai keluhan siny...,"[siang, kak, johan, maaf, keluh, sinyal, hilan..."
1770,0,nyala juga ni listrik bangun pagi gara listri...,"[nyala, ini, listrik, bangun, pagi, gara, list..."
1771,0,padam listrik berarti allah suruh lanjut rebah...,"[padam, listrik, allah, suruh, rebah]"


In [ ]:
def joinword(tweet):
  words = [word for word in tweet]
  return " ".join(words)

In [ ]:
dataTweet['TextProcessed'] = dataTweet['TextProcessed'].swifter.apply(joinword)

In [ ]:
dataTweet

,Label,Text,TextProcessed
0,1,Di Pepo mangkrak bangun jalan Non tol ?\nOtak ...,pepo mangkrak bangun jalan non tol otak mana o...
1,1,Pak ganjar hebat. Tp cuma untuk daerah2 yang p...,ganjar hebat tapi daerah kota daerah batas jat...
2,1,Kalo perbatasan bekasi jakarta mah satunya jal...,batas bekas jakarta mah satu jalan rusak loban...
3,1,Naik motor kena jalan gejlukan jalan yg rusak ...,motor kena jalan gejlukan jalan rusak sampai r...
4,1,Bangun tidur badan sakit sakit semuaaa grgr ja...,bangun tidur badan sakit sakit semua gara-gara...
...,...,...,...
1768,0,mencari sinyal dari bts yang lebih jauh dan t...,cari sinyal bts kondisi sinyal lemah banding s...
1769,0,siang kak johan maaf ya mengenai keluhan siny...,siang kak johan maaf keluh sinyal hilang listr...
1770,0,nyala juga ni listrik bangun pagi gara listri...,nyala ini listrik bangun pagi gara listrik pad...
1771,0,padam listrik berarti allah suruh lanjut rebah...,padam listrik allah suruh rebah


Keywords

In [ ]:
from collections import Counter
from itertools import chain
v = dataTweet['TextProcessed'].str.split().tolist()
c = Counter(chain.from_iterable(v))
dataTweet['TextProcessed'] = [[j for j in i if c[j] > 1] for i in v]

In [ ]:
dataTweet

,Label,Text,TextProcessed
0,1,Di Pepo mangkrak bangun jalan Non tol ?\nOtak ...,"[mangkrak, bangun, jalan, tol, otak, mana, ota..."
1,1,Pak ganjar hebat. Tp cuma untuk daerah2 yang p...,"[ganjar, hebat, tapi, daerah, kota, daerah, ba..."
2,1,Kalo perbatasan bekasi jakarta mah satunya jal...,"[batas, bekas, jakarta, mah, satu, jalan, rusa..."
3,1,Naik motor kena jalan gejlukan jalan yg rusak ...,"[motor, kena, jalan, jalan, rusak, sampai, rum..."
4,1,Bangun tidur badan sakit sakit semuaaa grgr ja...,"[bangun, tidur, badan, sakit, sakit, semua, ga..."
...,...,...,...
1768,0,mencari sinyal dari bts yang lebih jauh dan t...,"[cari, sinyal, bts, kondisi, sinyal, lemah, si..."
1769,0,siang kak johan maaf ya mengenai keluhan siny...,"[siang, kak, maaf, keluh, sinyal, hilang, list..."
1770,0,nyala juga ni listrik bangun pagi gara listri...,"[nyala, ini, listrik, bangun, pagi, gara, list..."
1771,0,padam listrik berarti allah suruh lanjut rebah...,"[padam, listrik, allah, suruh, rebah]"


In [ ]:
def words_counter(tweet):
    count = Counter()
    for text in tweet:
      for word in text:
        count[word] += 1
    return count

total_unique = len(words_counter(dataTweet.TextProcessed))

In [ ]:
print(words_counter(dataTweet.TextProcessed)) 
print(total_unique)

Counter({'mati': 1168, 'listrik': 1085, 'jalan': 495, 'air': 475, 'padam': 331, 'rusak': 295, 'sudah': 274, 'jam': 191, 'banget': 190, 'rumah': 163, 'enggak': 148, 'nyala': 139, 'pln': 133, 'daerah': 131, 'tolong': 131, 'sampai': 115, 'bolong': 113, 'mohon': 105, 'pas': 99, 'pagi': 89, 'kayak': 88, 'lampu': 85, 'pdam': 83, 'hidup': 82, 'pakai': 76, 'baik': 74, 'habis': 67, 'malam': 62, 'kasih': 59, 'kali': 59, 'terus': 59, 'siang': 56, 'kerja': 55, 'bagaimana': 54, 'mandi': 53, 'parah': 50, 'bisa': 46, 'panas': 45, 'info': 45, 'dari': 45, 'gara': 45, 'bantu': 45, 'orang': 44, 'sinyal': 44, 'tapi': 43, 'mata': 43, 'begitu': 42, 'lapor': 42, 'ada': 42, 'belum': 41, 'terima': 40, 'hp': 40, 'benar': 40, 'ganggu': 39, 'macet': 38, 'warga': 37, 'begini': 37, 'halo': 37, 'lagi': 35, 'minum': 35, 'alir': 34, 'hari': 33, 'langsung': 33, 'hujan': 32, 'tidur': 31, 'ku': 31, 'kec': 31, 'kota': 30, 'sore': 30, 'mulu': 29, 'kondisi': 29, 'ini': 29, 'desa': 28, 'selamat': 28, 'bayar': 28, 'madam': 28

In [ ]:
dataTweet

,Label,Text,TextProcessed
0,1,Di Pepo mangkrak bangun jalan Non tol ?\nOtak ...,"[mangkrak, bangun, jalan, tol, otak, mana, ota..."
1,1,Pak ganjar hebat. Tp cuma untuk daerah2 yang p...,"[ganjar, hebat, tapi, daerah, kota, daerah, ba..."
2,1,Kalo perbatasan bekasi jakarta mah satunya jal...,"[batas, bekas, jakarta, mah, satu, jalan, rusa..."
3,1,Naik motor kena jalan gejlukan jalan yg rusak ...,"[motor, kena, jalan, jalan, rusak, sampai, rum..."
4,1,Bangun tidur badan sakit sakit semuaaa grgr ja...,"[bangun, tidur, badan, sakit, sakit, semua, ga..."
...,...,...,...
1768,0,mencari sinyal dari bts yang lebih jauh dan t...,"[cari, sinyal, bts, kondisi, sinyal, lemah, si..."
1769,0,siang kak johan maaf ya mengenai keluhan siny...,"[siang, kak, maaf, keluh, sinyal, hilang, list..."
1770,0,nyala juga ni listrik bangun pagi gara listri...,"[nyala, ini, listrik, bangun, pagi, gara, list..."
1771,0,padam listrik berarti allah suruh lanjut rebah...,"[padam, listrik, allah, suruh, rebah]"


In [ ]:
def removeempty(tweet):
  res = [ele for ele in tweet if ele != []]
  return " ".join(res)

In [ ]:
dataTweet['TextREProcessed'] = dataTweet['TextProcessed'].swifter.apply(removeempty)

In [ ]:
dataTweet

,Label,Text,TextProcessed,TextREProcessed
0,1,Di Pepo mangkrak bangun jalan Non tol ?\nOtak ...,"[mangkrak, bangun, jalan, tol, otak, mana, ota...",mangkrak bangun jalan tol otak mana otak kasih...
1,1,Pak ganjar hebat. Tp cuma untuk daerah2 yang p...,"[ganjar, hebat, tapi, daerah, kota, daerah, ba...",ganjar hebat tapi daerah kota daerah batas jat...
2,1,Kalo perbatasan bekasi jakarta mah satunya jal...,"[batas, bekas, jakarta, mah, satu, jalan, rusa...",batas bekas jakarta mah satu jalan rusak loban...
3,1,Naik motor kena jalan gejlukan jalan yg rusak ...,"[motor, kena, jalan, jalan, rusak, sampai, rum...",motor kena jalan jalan rusak sampai rumah peru...
4,1,Bangun tidur badan sakit sakit semuaaa grgr ja...,"[bangun, tidur, badan, sakit, sakit, semua, ga...",bangun tidur badan sakit sakit semua gara-gara...
...,...,...,...,...
1768,0,mencari sinyal dari bts yang lebih jauh dan t...,"[cari, sinyal, bts, kondisi, sinyal, lemah, si...",cari sinyal bts kondisi sinyal lemah sila coba...
1769,0,siang kak johan maaf ya mengenai keluhan siny...,"[siang, kak, maaf, keluh, sinyal, hilang, list...",siang kak maaf keluh sinyal hilang listrik pad...
1770,0,nyala juga ni listrik bangun pagi gara listri...,"[nyala, ini, listrik, bangun, pagi, gara, list...",nyala ini listrik bangun pagi gara listrik pad...
1771,0,padam listrik berarti allah suruh lanjut rebah...,"[padam, listrik, allah, suruh, rebah]",padam listrik allah suruh rebah


In [ ]:
dataTweet['TextREProcessed'] = dataTweet['TextREProcessed'].swifter.apply(processTweet)

In [ ]:
dataTweet

,Label,Text,TextProcessed,TextREProcessed
0,1,Di Pepo mangkrak bangun jalan Non tol ?\nOtak ...,"[mangkrak, bangun, jalan, tol, otak, mana, ota...",mangkrak bangun jalan tol otak mana otak kasih...
1,1,Pak ganjar hebat. Tp cuma untuk daerah2 yang p...,"[ganjar, hebat, tapi, daerah, kota, daerah, ba...",ganjar hebat tapi daerah kota daerah batas jat...
2,1,Kalo perbatasan bekasi jakarta mah satunya jal...,"[batas, bekas, jakarta, mah, satu, jalan, rusa...",batas bekas jakarta mah satu jalan rusak loban...
3,1,Naik motor kena jalan gejlukan jalan yg rusak ...,"[motor, kena, jalan, jalan, rusak, sampai, rum...",motor kena jalan jalan rusak sampai rumah peru...
4,1,Bangun tidur badan sakit sakit semuaaa grgr ja...,"[bangun, tidur, badan, sakit, sakit, semua, ga...",bangun tidur badan sakit sakit semua garagara ...
...,...,...,...,...
1768,0,mencari sinyal dari bts yang lebih jauh dan t...,"[cari, sinyal, bts, kondisi, sinyal, lemah, si...",cari sinyal bts kondisi sinyal lemah sila coba...
1769,0,siang kak johan maaf ya mengenai keluhan siny...,"[siang, kak, maaf, keluh, sinyal, hilang, list...",siang kak maaf keluh sinyal hilang listrik pad...
1770,0,nyala juga ni listrik bangun pagi gara listri...,"[nyala, ini, listrik, bangun, pagi, gara, list...",nyala ini listrik bangun pagi gara listrik pad...
1771,0,padam listrik berarti allah suruh lanjut rebah...,"[padam, listrik, allah, suruh, rebah]",padam listrik allah suruh rebah


# **FastText**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from gensim.models import FastText
from gensim.test.utils import datapath

path = datapath('/content/drive/My Drive/cc.id.300.bin')
embedding = FastText.load_fasttext_format(path)

In [ ]:
embedding.most_similar('mati')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('mati.Bo', 0.7306226491928101),
 ('mati.Ia', 0.7183191180229187),
 ('mati.2', 0.7148823738098145),
 ('mati-', 0.7112743854522705),
 ('mati5', 0.7014772295951843),
 ('matinya', 0.6999092102050781),
 ('matii', 0.6971365213394165),
 ('mati.-', 0.6965159177780151),
 ('mati.', 0.6949059367179871),
 ('matian', 0.667380154132843)]

In [ ]:
embedding['makan'].shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


(300,)

In [ ]:
def get_vector(x):
  word_embedding = embedding[x]
  return word_embedding

In [ ]:
dataTweet

,Label,Text,TextProcessed,TextREProcessed
0,1,Di Pepo mangkrak bangun jalan Non tol ?\nOtak ...,"[mangkrak, bangun, jalan, tol, otak, mana, ota...",mangkrak bangun jalan tol otak mana otak kasih...
1,1,Pak ganjar hebat. Tp cuma untuk daerah2 yang p...,"[ganjar, hebat, tapi, daerah, kota, daerah, ba...",ganjar hebat tapi daerah kota daerah batas jat...
2,1,Kalo perbatasan bekasi jakarta mah satunya jal...,"[batas, bekas, jakarta, mah, satu, jalan, rusa...",batas bekas jakarta mah satu jalan rusak satu ...
3,1,Naik motor kena jalan gejlukan jalan yg rusak ...,"[motor, kena, jalan, jalan, rusak, sampai, rum...",motor kena jalan jalan rusak sampai rumah peru...
4,1,Bangun tidur badan sakit sakit semuaaa grgr ja...,"[bangun, tidur, badan, sakit, sakit, semua, ga...",bangun tidur badan sakit sakit semua gara-gara...
...,...,...,...,...
1257,0,di jalan jam 1 siang bolong matahari pontianak...,"[jalan, jam, siang, bolong, main, sampai, temp...",jalan jam siang bolong main sampai tempat kerj...
1258,0,@caratstalk @pledis_17 Memuji kegantengan send...,"[puji, ganteng, foto, kucing, beda, beda, jala...",puji ganteng foto kucing beda beda jalan siang...
1259,0,"Gila...jalan kaki di siang bolong, muka dah ke...","[kaki, siang, bolong, muka, sudah, kayak, samp...",kaki siang bolong muka sudah kayak sampai kaki...
1260,1,macet mulu air dasar ðŸ’© @ pdam,"[macet, mulu, air, dasar, pdam]",macet mulu air dasar pdam


# **CNN-Bidirectional**

In [ ]:
import numpy as np
import keras.backend as K

train_size = int(0.8*(len(dataTweet['TextREProcessed'])))
test_size = int(0.2*(len(dataTweet['TextREProcessed'])))

max_no_tokens = 40

indexes = set(np.random.choice(len(dataTweet['TextREProcessed']), train_size + test_size, replace=False))

x_train = np.zeros((train_size, max_no_tokens, 300), dtype=K.floatx())
y_train = np.zeros((train_size, 2), dtype=np.int32)

x_test = np.zeros((test_size, max_no_tokens, 300), dtype=K.floatx())
y_test = np.zeros((test_size, 2), dtype=np.int32)

In [ ]:
for i, index in enumerate(indexes):
    for t, token in enumerate(dataTweet['TextREProcessed'][index]):
        if t >= max_no_tokens:
            break
      
        if token not in embedding.wv:
            continue
    
        if i < train_size:
            x_train[i, t, :] = embedding.wv[token]
        else:
            x_test[i - train_size, t, :] = embedding.wv[token]

  
    if i < train_size:
        y_train[i, :] = [1.0, 0.0] if dataTweet['Label'][index] == 0 else [0.0, 1.0]
    else:
        y_test[i - train_size, :] = [1.0, 0.0] if dataTweet['Label'][index] == 0 else [0.0, 1.0]

In [ ]:
x_train.shape, y_test.shape

((1418, 40, 300), (354, 2))

In [ ]:
from keras.models import Sequential
from keras.layers import Conv1D, Dropout, Dense, Flatten, LSTM, MaxPooling1D, Bidirectional
from keras.optimizers import Adam

model = Sequential()

model.add(Conv1D(32, kernel_size=3, activation='elu', padding='same',
                 input_shape=(max_no_tokens, 300)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(16, kernel_size=3, activation='relu', padding='same'))
model.add(MaxPooling1D(pool_size=2))
model.add(Bidirectional(LSTM(512, dropout=0.1, recurrent_dropout=0.3)))

model.add(Dropout(0.25))

model.add(Dense(2, activation='sigmoid'))

model.compile(loss='BinaryCrossentropy', optimizer=Adam(lr=0.0005), metrics=['accuracy'])

model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_12 (Conv1D)           (None, 40, 32)            28832     
_________________________________________________________________
max_pooling1d_11 (MaxPooling (None, 20, 32)            0         
_________________________________________________________________
conv1d_13 (Conv1D)           (None, 20, 16)            1552      
_________________________________________________________________
max_pooling1d_12 (MaxPooling (None, 10, 16)            0         
_________________________________________________________________
bidirectional_8 (Bidirection (None, 1024)              2166784   
_________________________________________________________________
dropout_8 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 2)                

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
no_epochs = 10

In [ ]:
model.fit(x_train, y_train, shuffle=True, epochs=no_epochs,
         validation_data=(x_test, y_test))

Epoch 1/10
45/45 [==============================] - 32s 546ms/step - loss: 0.6948 - accuracy: 0.4722 - val_loss: 0.6888 - val_accuracy: 0.5706
Epoch 2/10
45/45 [==============================] - 23s 505ms/step - loss: 0.6842 - accuracy: 0.5830 - val_loss: 0.6855 - val_accuracy: 0.5339
Epoch 3/10
45/45 [==============================] - 23s 520ms/step - loss: 0.6681 - accuracy: 0.5972 - val_loss: 0.6852 - val_accuracy: 0.5537
Epoch 4/10
45/45 [==============================] - 23s 508ms/step - loss: 0.6318 - accuracy: 0.6341 - val_loss: 0.6723 - val_accuracy: 0.6130
Epoch 5/10
45/45 [==============================] - 24s 523ms/step - loss: 0.5672 - accuracy: 0.7222 - val_loss: 0.7770 - val_accuracy: 0.6186
Epoch 6/10
45/45 [==============================] - 23s 518ms/step - loss: 0.5242 - accuracy: 0.7466 - val_loss: 0.6415 - val_accuracy: 0.6215
Epoch 7/10
45/45 [==============================] - 23s 501ms/step - loss: 0.4835 - accuracy: 0.7737 - val_loss: 0.7174 - val_accuracy: 0.5763

In [ ]:
newdataTweet = pd.read_csv('Layani_Dataset.csv')

In [ ]:
newdataTweet['TextProcessed'] = newdataTweet['Text'].apply(processTweet)

In [ ]:
newdataTweet['TextProcessed'] = newdataTweet['TextProcessed'].swifter.apply(remove_stopwords)

In [ ]:
newdataTweet['TextProcessed'] = newdataTweet['TextProcessed'].swifter.apply(normalized_term)

In [ ]:
for tweet in newdataTweet['TextProcessed']:
    for word in tweet:
        if word not in word_dict:
            word_dict[word] = ' '            

for word in word_dict:
    word_dict[word] = stemmed_wrapper(word)

In [ ]:
newdataTweet['TextProcessed'] = newdataTweet['TextProcessed'].swifter.apply(get_stemmed_term)

In [ ]:
predict_size = int((len(newdataTweet['TextProcessed'])))

max_no_tokens = 40

indexes = set(np.random.choice(len(newdataTweet['Text']), predict_size, replace=False))

x_predict = np.zeros((predict_size, max_no_tokens, 300), dtype=K.floatx())

In [ ]:
for i, index in enumerate(indexes):
    for t, token in enumerate(newdataTweet['TextProcessed'][index]):
        if t >= max_no_tokens:
            break
      
        if token not in ft_model.wv:
            continue

        x_predict[i, t, :] = ft_model.wv[token]

In [ ]:
x_predict.shape

In [ ]:
prediction = model.predict(x_predict)
predlabel = []
complaint = []
false = 0
for i in range(len(x_predict)):
  if prediction[i][0] < prediction[i][1]:
    complaint.append(newdataTweet['Text'][i])
    predlabel.append(1)
  else:
    predlabel.append(0)
    
  if predlabel[i] != newdataTweet["Label"][i]:
    false +=1
acc = 1 - false/len(x_predict)
print("Model Acc = ", acc, "\n")
print(complaint)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers
from keras.models import load_model

In [ ]:
def split_data(dataset):
  dataset = dataset.sample(frac=1, random_state=42)
  train_data, val_data = dataset[80:], dataset[:80]
  train_tweets = train_data.Text.to_numpy()
  train_labels = train_data.Label.to_numpy()
  train_vec = train_data.Vec.to_numpy()
  val_tweets = val_data.Text.to_numpy()
  val_labels = val_data.Label.to_numpy()
  val_vec = val_data.Vec.to_numpy()
  return train_data, train_tweets, train_labels, train_vec, val_data, val_tweets, val_labels, val_vec

train_data, train_tweets, train_labels, train_vec, val_data, val_tweets, val_labels, val_vec = split_data(dataTweet)
#test_tweets = new_dataTweet.Text.to_numpy()

In [ ]:
val_vec = val_vec.reshape(-1,1)
train_vec = train_vec.reshape(-1,1)
val_vec = np.concatenate(np.concatenate(val_vec, axis = 0), axis = 0)
train_vec = np.concatenate(np.concatenate(train_vec, axis = 0), axis = 0)

print(val_vec.shape)
print(train_vec.shape)

In [ ]:
def embedding(text):
  tokenizer = Tokenizer(num_words = total_unique)
  tokenizer.fit_on_texts(text)
  word_index = tokenizer.word_index
  text_sequences = tokenizer.texts_to_sequences(text)
  return text_sequences

def padding(text_seq):
  text_padded = pad_sequences(text_seq, maxlen=40, padding="post", truncating="post")
  return text_padded

In [ ]:
train_sequences = embedding(train_tweets)
train_padded = padding(train_sequences)
val_sequences = embedding(val_tweets)
val_padded = padding(val_sequences)
#test_sequences = embedding(test_tweets)
#test_padded = padding(test_sequences)

In [ ]:
model = keras.models.Sequential()
model.add(layers.Embedding(total_unique, 32, input_length=40))
model.add(layers.LSTM(64, dropout=0.1))
model.add(layers.Dense(1, activation='sigmoid'))

loss = keras.losses.BinaryCrossentropy(from_logits=False)
optim = keras.optimizers.Adam(learning_rate=0.0005)
metrics = ["accuracy"]

model.compile(loss=loss, optimizer=optim, metrics=metrics)

model.fit(train_vec, train_labels, epochs=10, validation_data=(val_vec, val_labels), verbose=2)